# Chapter 33: TypeScript with Node.js

Node.js has become the standard runtime for building scalable server-side applications with JavaScript. When combined with TypeScript, it provides type safety, better IDE support, and fewer runtime errors. This chapter covers everything from basic setup to advanced patterns for building production-ready Node.js applications with TypeScript.

---

## 33.1 Setting Up Node.js with TypeScript

Modern Node.js development with TypeScript requires proper tooling for compilation, development, and production builds.

### Project Initialization

```bash
# Create project directory
mkdir node-ts-app
cd node-ts-app

# Initialize package.json
npm init -y

# Install TypeScript and Node.js types
npm install -D typescript @types/node ts-node

# Install development tools
npm install -D nodemon tsx

# Initialize TypeScript configuration
npx tsc --init
```

**Recommended tsconfig.json for Node.js:**

```json
{
  "compilerOptions": {
    "target": "ES2022",
    "module": "NodeNext",
    "moduleResolution": "NodeNext",
    "lib": ["ES2022"],
    "outDir": "./dist",
    "rootDir": "./src",
    "strict": true,
    "esModuleInterop": true,
    "skipLibCheck": true,
    "forceConsistentCasingInFileNames": true,
    "resolveJsonModule": true,
    "declaration": true,
    "declarationMap": true,
    "sourceMap": true,
    "removeComments": true,
    "noImplicitAny": true,
    "strictNullChecks": true,
    "strictFunctionTypes": true,
    "noImplicitThis": true,
    "noUnusedLocals": true,
    "noUnusedParameters": true,
    "noImplicitReturns": true,
    "noFallthroughCasesInSwitch": true,
    "allowSyntheticDefaultImports": true,
    "experimentalDecorators": true,
    "emitDecoratorMetadata": true,
    "isolatedModules": true,
    "baseUrl": ".",
    "paths": {
      "@/*": ["src/*"],
      "@config/*": ["src/config/*"],
      "@services/*": ["src/services/*"],
      "@controllers/*": ["src/controllers/*"],
      "@middleware/*": ["src/middleware/*"],
      "@types/*": ["src/types/*"],
      "@utils/*": ["src/utils/*"]
    }
  },
  "include": ["src/**/*"],
  "exclude": ["node_modules", "dist", "**/*.test.ts", "**/*.spec.ts"]
}
```

**Explanation:**
- `module: "NodeNext"`: Uses modern ES modules with Node.js resolution
- `paths`: Configures path aliases for cleaner imports
- `outDir` and `rootDir`: Separates source and compiled code
- `declaration`: Generates `.d.ts` files for library consumers
- `sourceMap`: Enables debugging of TypeScript source

### Development Scripts

**package.json configuration:**

```json
{
  "name": "node-ts-app",
  "version": "1.0.0",
  "type": "module",
  "scripts": {
    "dev": "tsx watch src/index.ts",
    "build": "tsc",
    "start": "node dist/index.js",
    "start:prod": "npm run build && npm start",
    "type-check": "tsc --noEmit",
    "clean": "rm -rf dist"
  },
  "dependencies": {
    "dotenv": "^16.3.1"
  },
  "devDependencies": {
    "@types/node": "^20.10.0",
    "tsx": "^4.7.0",
    "typescript": "^5.3.0"
  }
}
```

**Explanation:**
- `type: "module"`: Enables ES modules (import/export) in Node.js
- `tsx`: Fast TypeScript execution (faster than ts-node, no type checking during dev)
- `tsx watch`: Automatically restarts on file changes (replaces nodemon)
- `tsc`: Production build compiles to JavaScript in `dist/`

### Basic Application Structure

```
node-ts-app/
├── src/
│   ├── config/
│   │   └── database.ts
│   ├── controllers/
│   │   └── userController.ts
│   ├── middleware/
│   │   └── errorHandler.ts
│   ├── routes/
│   │   └── userRoutes.ts
│   ├── services/
│   │   └── userService.ts
│   ├── types/
│   │   └── index.ts
│   ├── utils/
│   │   └── logger.ts
│   └── index.ts
├── .env
├── .env.example
├── .gitignore
├── package.json
└── tsconfig.json
```

**Entry point (src/index.ts):**

```typescript
import { createServer } from 'http';
import { config } from './config/environment.js';

const PORT = config.PORT || 3000;

const server = createServer((req, res) => {
  res.writeHead(200, { 'Content-Type': 'application/json' });
  res.end(JSON.stringify({ message: 'Hello from TypeScript Node.js!' }));
});

server.listen(PORT, () => {
  console.log(`Server running on port ${PORT}`);
});
```

---

## 33.2 Typing Express Applications

Express is the most popular Node.js web framework. TypeScript adds type safety to routes, middleware, and request/response handling.

### Basic Express Setup

```typescript
import express, { 
  Application, 
  Request, 
  Response, 
  NextFunction,
  RequestHandler,
  ErrorRequestHandler
} from 'express';

const app: Application = express();

// Middleware
app.use(express.json());
app.use(express.urlencoded({ extended: true }));

// Typed route handler
app.get('/health', (req: Request, res: Response) => {
  res.json({ status: 'ok', timestamp: new Date().toISOString() });
});

// Start server
const PORT = process.env.PORT || 3000;
app.listen(PORT, () => {
  console.log(`Server running on port ${PORT}`);
});
```

### Typed Request and Response

```typescript
import { Request, Response } from 'express';

// Define request body type
interface CreateUserRequest {
  email: string;
  password: string;
  name: string;
  age?: number;
}

// Define response type
interface UserResponse {
  id: string;
  email: string;
  name: string;
  createdAt: string;
}

// Define route parameters
interface UserParams {
  userId: string;
}

// Define query parameters
interface UserQuery {
  page?: string;
  limit?: string;
  sort?: 'asc' | 'desc';
}

// POST /users - Create user with typed body
app.post('/users', (req: Request<{}, UserResponse, CreateUserRequest>, res: Response<UserResponse>) => {
  const { email, password, name, age } = req.body;
  
  // TypeScript knows the types of these properties
  const newUser: UserResponse = {
    id: crypto.randomUUID(),
    email,
    name,
    createdAt: new Date().toISOString()
  };
  
  res.status(201).json(newUser);
});

// GET /users/:userId - Typed params
app.get('/users/:userId', (req: Request<UserParams>, res: Response<UserResponse>) => {
  const { userId } = req.params; // TypeScript knows this is string
  
  // Fetch user logic here
  const user: UserResponse = {
    id: userId,
    email: 'user@example.com',
    name: 'John Doe',
    createdAt: new Date().toISOString()
  };
  
  res.json(user);
});

// GET /users - Typed query params
app.get('/users', (req: Request<{}, {}, {}, UserQuery>, res: Response) => {
  const page = parseInt(req.query.page || '1');
  const limit = parseInt(req.query.limit || '10');
  const sort = req.query.sort || 'asc';
  
  // TypeScript ensures sort is 'asc' | 'desc' | undefined
  res.json({ page, limit, sort });
});
```

**Explanation:**
- `Request` generic: `<Params, ResBody, ReqBody, ReqQuery>`
- `Response` generic: `<ResBody, Locals>` (Locals for template variables)
- TypeScript validates that you're accessing the correct properties on `req.body`, `req.params`, etc.

### Custom Request Types with Augmentation

When adding custom properties to requests (e.g., user authentication):

```typescript
import { Request } from 'express';

// Extend Express Request type globally
declare global {
  namespace Express {
    interface Request {
      user?: {
        id: string;
        email: string;
        role: 'admin' | 'user' | 'guest';
      };
      requestId: string;
      startTime: number;
    }
  }
}

// Middleware that adds user to request
const authenticate: RequestHandler = (req, res, next) => {
  // Verify JWT token logic here
  req.user = {
    id: '123',
    email: 'user@example.com',
    role: 'user'
  };
  req.requestId = crypto.randomUUID();
  req.startTime = Date.now();
  
  next();
};

// Protected route using augmented request
app.get('/profile', authenticate, (req: Request, res: Response) => {
  // TypeScript knows req.user exists here (you may need type guard)
  if (!req.user) {
    return res.status(401).json({ error: 'Unauthorized' });
  }
  
  res.json({
    user: req.user,
    requestId: req.requestId
  });
});
```

### Typed Middleware

```typescript
import { Request, Response, NextFunction, RequestHandler } from 'express';

// Generic middleware type helper
type AsyncRequestHandler = (
  req: Request,
  res: Response,
  next: NextFunction
) => Promise<void>;

// Error middleware type
type ErrorMiddleware = (
  err: Error,
  req: Request,
  res: Response,
  next: NextFunction
) => void;

// Logging middleware
const requestLogger: RequestHandler = (req, res, next) => {
  const timestamp = new Date().toISOString();
  console.log(`[${timestamp}] ${req.method} ${req.path}`);
  next();
};

// Async error wrapper (prevents try-catch repetition)
const asyncHandler = (fn: AsyncRequestHandler): RequestHandler => {
  return (req, res, next) => {
    Promise.resolve(fn(req, res, next)).catch(next);
  };
};

// Validation middleware with generics
interface ValidationSchema<T> {
  validate: (data: unknown) => { error?: { message: string }; value: T };
}

const validateRequest = <T>(schema: ValidationSchema<T>): RequestHandler => {
  return (req, res, next) => {
    const { error, value } = schema.validate(req.body);
    
    if (error) {
      return res.status(400).json({ error: error.message });
    }
    
    // Replace req.body with validated value
    req.body = value;
    next();
  };
};

// Rate limiting middleware
interface RateLimitOptions {
  windowMs: number;
  maxRequests: number;
}

const rateLimit = (options: RateLimitOptions): RequestHandler => {
  const requests = new Map<string, number[]>();
  
  return (req, res, next) => {
    const key = req.ip || 'unknown';
    const now = Date.now();
    const windowStart = now - options.windowMs;
    
    const userRequests = requests.get(key) || [];
    const recentRequests = userRequests.filter(time => time > windowStart);
    
    if (recentRequests.length >= options.maxRequests) {
      return res.status(429).json({ error: 'Too many requests' });
    }
    
    recentRequests.push(now);
    requests.set(key, recentRequests);
    next();
  };
};

// Error handling middleware
const errorHandler: ErrorMiddleware = (err, req, res, next) => {
  console.error(err.stack);
  
  if (err.name === 'ValidationError') {
    return res.status(400).json({ error: err.message });
  }
  
  if (err.name === 'UnauthorizedError') {
    return res.status(401).json({ error: 'Unauthorized' });
  }
  
  res.status(500).json({ error: 'Internal server error' });
};

// Usage
app.use(requestLogger);
app.use(rateLimit({ windowMs: 60000, maxRequests: 100 }));
app.post('/users', validateRequest(userSchema), createUser);
app.use(errorHandler);
```

### Controller Pattern with TypeScript

```typescript
// types.ts
export interface CreateUserDTO {
  email: string;
  password: string;
  name: string;
}

export interface UpdateUserDTO {
  email?: string;
  name?: string;
  age?: number;
}

export interface UserResponse {
  id: string;
  email: string;
  name: string;
  createdAt: Date;
  updatedAt: Date;
}

// Base controller interface
interface Controller {
  create: RequestHandler;
  findAll: RequestHandler;
  findOne: RequestHandler;
  update: RequestHandler;
  remove: RequestHandler;
}

// UserController.ts
import { Request, Response } from 'express';
import { UserService } from '../services/UserService.js';
import { CreateUserDTO, UpdateUserDTO, UserResponse } from '../types/index.js';

export class UserController implements Controller {
  constructor(private userService: UserService) {}

  create = async (req: Request<{}, UserResponse, CreateUserDTO>, res: Response<UserResponse | { error: string }>) => {
    try {
      const user = await this.userService.create(req.body);
      res.status(201).json(user);
    } catch (error) {
      res.status(400).json({ error: (error as Error).message });
    }
  };

  findAll = async (req: Request, res: Response<UserResponse[]>) => {
    const users = await this.userService.findAll();
    res.json(users);
  };

  findOne = async (req: Request<{ id: string }>, res: Response<UserResponse | { error: string }>) => {
    try {
      const user = await this.userService.findOne(req.params.id);
      if (!user) {
        return res.status(404).json({ error: 'User not found' });
      }
      res.json(user);
    } catch (error) {
      res.status(500).json({ error: (error as Error).message });
    }
  };

  update = async (req: Request<{ id: string }, {}, UpdateUserDTO>, res: Response<UserResponse | { error: string }>) => {
    try {
      const user = await this.userService.update(req.params.id, req.body);
      res.json(user);
    } catch (error) {
      res.status(400).json({ error: (error as Error).message });
    }
  };

  remove = async (req: Request<{ id: string }>, res: Response<void | { error: string }>) => {
    try {
      await this.userService.remove(req.params.id);
      res.status(204).send();
    } catch (error) {
      res.status(500).json({ error: (error as Error).message });
    }
  };
}
```

---

## 33.3 Typing Fastify Applications

Fastify is a high-performance framework with built-in JSON Schema validation. TypeScript integration is excellent with the `fastify-type-provider-typebox` or `fastify-type-provider-zod` packages.

### Basic Fastify Setup

```typescript
import Fastify, { FastifyInstance, FastifyRequest, FastifyReply } from 'fastify';

const app: FastifyInstance = Fastify({
  logger: true
});

// Typed route
app.get('/health', async (request: FastifyRequest, reply: FastifyReply) => {
  return { status: 'ok', timestamp: new Date().toISOString() };
});

// Start server
const start = async () => {
  try {
    await app.listen({ port: 3000 });
    console.log('Server running on port 3000');
  } catch (err) {
    app.log.error(err);
    process.exit(1);
  }
};

start();
```

### JSON Schema Validation with TypeBox

TypeBox allows you to define JSON Schema and TypeScript interfaces simultaneously:

```typescript
import Fastify from 'fastify';
import { TypeBoxTypeProvider } from '@fastify/type-provider-typebox';
import { Type } from '@sinclair/typebox';

// Create app with TypeBox type provider
const app = Fastify().withTypeProvider<TypeBoxTypeProvider>();

// Define schema and types together
const UserSchema = Type.Object({
  id: Type.String(),
  email: Type.String({ format: 'email' }),
  name: Type.String(),
  age: Type.Optional(Type.Number({ minimum: 0, maximum: 150 }))
});

const CreateUserSchema = Type.Object({
  email: Type.String({ format: 'email' }),
  password: Type.String({ minLength: 8 }),
  name: Type.String(),
  age: Type.Optional(Type.Number())
});

// Infer TypeScript types from schemas
type User = typeof UserSchema.static;
type CreateUserDTO = typeof CreateUserSchema.static;

// POST route with validation
app.post('/users', {
  schema: {
    body: CreateUserSchema,
    response: {
      201: UserSchema
    }
  }
}, async (request, reply) => {
  // request.body is fully typed as CreateUserDTO
  const { email, password, name, age } = request.body;
  
  const newUser: User = {
    id: crypto.randomUUID(),
    email,
    name,
    age
  };
  
  reply.status(201);
  return newUser;
});

// GET route with params
const UserParamsSchema = Type.Object({
  userId: Type.String()
});

app.get('/users/:userId', {
  schema: {
    params: UserParamsSchema
  }
}, async (request, reply) => {
  // request.params is typed as { userId: string }
  const { userId } = request.params;
  
  return {
    id: userId,
    email: 'user@example.com',
    name: 'John Doe'
  };
});
```

**Explanation:**
- TypeBox generates both JSON Schema (for validation) and TypeScript types
- `withTypeProvider<TypeBoxTypeProvider>()` enables type inference from schemas
- `request.body`, `request.params`, `request.query` are automatically typed
- No manual interface definitions needed - single source of truth

### Hooks and Decorators

```typescript
import fp from 'fastify-plugin';

// Custom decorator
declare module 'fastify' {
  interface FastifyInstance {
    authenticate: () => Promise<void>;
    config: {
      JWT_SECRET: string;
      DATABASE_URL: string;
    };
  }
  
  interface FastifyRequest {
    user: {
      id: string;
      email: string;
      role: string;
    };
  }
}

// Authentication hook
app.addHook('onRequest', async (request, reply) => {
  // Skip auth for public routes
  if (request.url === '/login' || request.url === '/register') {
    return;
  }
  
  try {
    const token = request.headers.authorization?.replace('Bearer ', '');
    if (!token) {
      throw new Error('No token provided');
    }
    
    // Verify JWT
    const decoded = await verifyToken(token);
    request.user = decoded;
  } catch (err) {
    reply.status(401).send({ error: 'Unauthorized' });
  }
});

// Plugin with TypeScript
const databasePlugin = fp(async (fastify, options) => {
  const db = await createDatabaseConnection(options.connectionString);
  
  fastify.decorate('db', db);
  
  fastify.addHook('onClose', async (instance) => {
    await db.close();
  });
});

// Register plugin
app.register(databasePlugin, { connectionString: process.env.DATABASE_URL });

// Typed route using decorated properties
app.get('/protected', async (request, reply) => {
  // TypeScript knows request.user exists
  return {
    message: `Hello ${request.user.email}`,
    userId: request.user.id
  };
});
```

---

## 33.4 Typing NestJS Applications

NestJS is a progressive Node.js framework built with TypeScript in mind. It uses decorators, dependency injection, and modular architecture similar to Angular.

### Module and Controller Setup

```typescript
// main.ts
import { NestFactory } from '@nestjs/core';
import { AppModule } from './app.module.js';
import { ValidationPipe } from '@nestjs/common';

async function bootstrap() {
  const app = await NestFactory.create(AppModule);
  
  // Global validation pipe
  app.useGlobalPipes(new ValidationPipe({
    whitelist: true,
    forbidNonWhitelisted: true,
    transform: true
  }));
  
  await app.listen(3000);
}
bootstrap();

// app.module.ts
import { Module } from '@nestjs/common';
import { UsersModule } from './users/users.module.js';
import { DatabaseModule } from './database/database.module.js';

@Module({
  imports: [DatabaseModule, UsersModule],
  controllers: [],
  providers: [],
})
export class AppModule {}

// users/users.module.ts
import { Module } from '@nestjs/common';
import { UsersController } from './users.controller.js';
import { UsersService } from './users.service.js';

@Module({
  controllers: [UsersController],
  providers: [UsersService],
  exports: [UsersService]
})
export class UsersModule {}
```

### DTOs and Validation

```typescript
// create-user.dto.ts
import { IsEmail, IsString, MinLength, IsOptional, IsInt, Min, Max } from 'class-validator';

export class CreateUserDto {
  @IsEmail()
  email: string;

  @IsString()
  @MinLength(8)
  password: string;

  @IsString()
  name: string;

  @IsOptional()
  @IsInt()
  @Min(0)
  @Max(150)
  age?: number;
}

// update-user.dto.ts
import { PartialType } from '@nestjs/mapped-types';
import { CreateUserDto } from './create-user.dto.js';

export class UpdateUserDto extends PartialType(CreateUserDto) {}

// user.entity.ts
export class User {
  id: string;
  email: string;
  password: string;
  name: string;
  age?: number;
  createdAt: Date;
  updatedAt: Date;
}
```

### Controller with TypeScript

```typescript
// users.controller.ts
import {
  Controller,
  Get,
  Post,
  Body,
  Patch,
  Param,
  Delete,
  Query,
  ParseIntPipe,
  DefaultValuePipe,
  HttpCode,
  HttpStatus
} from '@nestjs/common';
import { UsersService } from './users.service.js';
import { CreateUserDto } from './dto/create-user.dto.js';
import { UpdateUserDto } from './dto/update-user.dto.js';
import { User } from './entities/user.entity.js';

@Controller('users')
export class UsersController {
  constructor(private readonly usersService: UsersService) {}

  @Post()
  @HttpCode(HttpStatus.CREATED)
  async create(@Body() createUserDto: CreateUserDto): Promise<User> {
    return this.usersService.create(createUserDto);
  }

  @Get()
  async findAll(
    @Query('page', new DefaultValuePipe(1), ParseIntPipe) page: number,
    @Query('limit', new DefaultValuePipe(10), ParseIntPipe) limit: number
  ): Promise<{ data: User[]; total: number }> {
    return this.usersService.findAll({ page, limit });
  }

  @Get(':id')
  async findOne(@Param('id') id: string): Promise<User> {
    return this.usersService.findOne(id);
  }

  @Patch(':id')
  async update(
    @Param('id') id: string,
    @Body() updateUserDto: UpdateUserDto
  ): Promise<User> {
    return this.usersService.update(id, updateUserDto);
  }

  @Delete(':id')
  @HttpCode(HttpStatus.NO_CONTENT)
  async remove(@Param('id') id: string): Promise<void> {
    return this.usersService.remove(id);
  }
}
```

### Service Layer with Dependency Injection

```typescript
// users.service.ts
import { Injectable, NotFoundException, ConflictException } from '@nestjs/common';
import { CreateUserDto } from './dto/create-user.dto.js';
import { UpdateUserDto } from './dto/update-user.dto.js';
import { User } from './entities/user.entity.js';
import { DatabaseService } from '../database/database.service.js';

interface FindAllOptions {
  page: number;
  limit: number;
}

@Injectable()
export class UsersService {
  constructor(private db: DatabaseService) {}

  async create(createUserDto: CreateUserDto): Promise<User> {
    // Check if email exists
    const existing = await this.db.users.findByEmail(createUserDto.email);
    if (existing) {
      throw new ConflictException('Email already exists');
    }
    
    const user = await this.db.users.create({
      ...createUserDto,
      id: crypto.randomUUID(),
      createdAt: new Date(),
      updatedAt: new Date()
    });
    
    return user;
  }

  async findAll(options: FindAllOptions): Promise<{ data: User[]; total: number }> {
    const skip = (options.page - 1) * options.limit;
    const [data, total] = await Promise.all([
      this.db.users.findMany({ skip, take: options.limit }),
      this.db.users.count()
    ]);
    
    return { data, total };
  }

  async findOne(id: string): Promise<User> {
    const user = await this.db.users.findById(id);
    if (!user) {
      throw new NotFoundException(`User with ID ${id} not found`);
    }
    return user;
  }

  async update(id: string, updateUserDto: UpdateUserDto): Promise<User> {
    await this.findOne(id); // Ensure user exists
    
    return this.db.users.update(id, {
      ...updateUserDto,
      updatedAt: new Date()
    });
  }

  async remove(id: string): Promise<void> {
    await this.findOne(id);
    await this.db.users.delete(id);
  }
}
```

### Guards and Interceptors

```typescript
// guards/auth.guard.ts
import { Injectable, CanActivate, ExecutionContext, UnauthorizedException } from '@nestjs/common';
import { Observable } from 'rxjs';
import { Request } from 'express';

interface AuthenticatedRequest extends Request {
  user: {
    id: string;
    email: string;
    role: string;
  };
}

@Injectable()
export class AuthGuard implements CanActivate {
  canActivate(
    context: ExecutionContext,
  ): boolean | Promise<boolean> | Observable<boolean> {
    const request = context.switchToHttp().getRequest<AuthenticatedRequest>();
    const token = request.headers.authorization?.split(' ')[1];
    
    if (!token) {
      throw new UnauthorizedException();
    }
    
    try {
      const payload = verifyToken(token);
      request.user = payload;
      return true;
    } catch {
      throw new UnauthorizedException();
    }
  }
}

// decorators/roles.decorator.ts
import { SetMetadata } from '@nestjs/common';

export const ROLES_KEY = 'roles';
export const Roles = (...roles: string[]) => SetMetadata(ROLES_KEY, roles);

// guards/roles.guard.ts
import { Injectable, CanActivate, ExecutionContext } from '@nestjs/common';
import { Reflector } from '@nestjs/core';

@Injectable()
export class RolesGuard implements CanActivate {
  constructor(private reflector: Reflector) {}

  canActivate(context: ExecutionContext): boolean {
    const requiredRoles = this.reflector.getAllAndOverride<string[]>(ROLES_KEY, [
      context.getHandler(),
      context.getClass(),
    ]);
    
    if (!requiredRoles) {
      return true;
    }
    
    const { user } = context.switchToHttp().getRequest();
    return requiredRoles.some((role) => user.role === role);
  }
}

// Usage in controller
@Controller('admin')
@UseGuards(AuthGuard, RolesGuard)
export class AdminController {
  @Get('users')
  @Roles('admin')
  findAll() {
    // Only accessible to admins
  }
}
```

---

## 33.5 Database Integration Types

Type safety extends to database operations through ORMs and query builders.

### Prisma ORM

Prisma generates TypeScript types from your database schema:

```typescript
// schema.prisma
generator client {
  provider = "prisma-client-js"
}

datasource db {
  provider = "postgresql"
  url      = env("DATABASE_URL")
}

model User {
  id        String   @id @default(uuid())
  email     String   @unique
  password  String
  name      String
  age       Int?
  role      Role     @default(USER)
  posts     Post[]
  createdAt DateTime @default(now())
  updatedAt DateTime @updatedAt
}

model Post {
  id       String @id @default(uuid())
  title    String
  content  String?
  published Boolean @default(false)
  author   User   @relation(fields: [authorId], references: [id])
  authorId String
}

enum Role {
  USER
  ADMIN
  MODERATOR
}

// Generated types (prisma/client)
import { PrismaClient, User, Post, Role } from '@prisma/client';

const prisma = new PrismaClient();

// Fully typed CRUD operations
async function createUser(data: {
  email: string;
  password: string;
  name: string;
  age?: number;
}): Promise<User> {
  return prisma.user.create({ data });
}

async function getUserWithPosts(id: string): Promise<User & { posts: Post[] } | null> {
  return prisma.user.findUnique({
    where: { id },
    include: { posts: true }
  });
}

async function updateUser(
  id: string, 
  data: Partial<Pick<User, 'email' | 'name' | 'age'>>
): Promise<User> {
  return prisma.user.update({
    where: { id },
    data
  });
}

// Type-safe queries
const users = await prisma.user.findMany({
  where: {
    age: { gte: 18 },
    role: Role.ADMIN
  },
  select: {
    id: true,
    email: true,
    name: true
  }
});
// TypeScript knows users is Array<{ id: string; email: string; name: string }>
```

### TypeORM

```typescript
import { Entity, PrimaryGeneratedColumn, Column, CreateDateColumn, UpdateDateColumn, OneToMany } from 'typeorm';

@Entity()
export class User {
  @PrimaryGeneratedColumn('uuid')
  id: string;

  @Column({ unique: true })
  email: string;

  @Column()
  password: string;

  @Column()
  name: string;

  @Column({ nullable: true })
  age?: number;

  @Column({ type: 'enum', enum: ['user', 'admin'], default: 'user' })
  role: 'user' | 'admin';

  @OneToMany(() => Post, post => post.author)
  posts: Post[];

  @CreateDateColumn()
  createdAt: Date;

  @UpdateDateColumn()
  updatedAt: Date;
}

// Repository pattern
import { Repository } from 'typeorm';
import { Injectable } from '@nestjs/common';
import { InjectRepository } from '@nestjs/typeorm';

@Injectable()
export class UsersService {
  constructor(
    @InjectRepository(User)
    private usersRepository: Repository<User>
  ) {}

  async findAll(): Promise<User[]> {
    return this.usersRepository.find();
  }

  async findOne(id: string): Promise<User | null> {
    return this.usersRepository.findOne({ where: { id } });
  }

  async create(userData: Partial<User>): Promise<User> {
    const user = this.usersRepository.create(userData);
    return this.usersRepository.save(user);
  }
}
```

### Drizzle ORM (Lightweight Alternative)

```typescript
import { pgTable, uuid, varchar, integer, timestamp, text, boolean } from 'drizzle-orm/pg-core';
import { eq } from 'drizzle-orm';
import { drizzle } from 'drizzle-orm/node-postgres';
import { Pool } from 'pg';

// Schema definition
export const users = pgTable('users', {
  id: uuid('id').primaryKey().defaultRandom(),
  email: varchar('email', { length: 255 }).notNull().unique(),
  password: varchar('password', { length: 255 }).notNull(),
  name: varchar('name', { length: 255 }).notNull(),
  age: integer('age'),
  createdAt: timestamp('created_at').defaultNow().notNull(),
  updatedAt: timestamp('updated_at').defaultNow().notNull()
});

export const posts = pgTable('posts', {
  id: uuid('id').primaryKey().defaultRandom(),
  title: varchar('title', { length: 255 }).notNull(),
  content: text('content'),
  published: boolean('published').default(false),
  authorId: uuid('author_id').references(() => users.id).notNull(),
  createdAt: timestamp('created_at').defaultNow().notNull()
});

// Types inferred from schema
type User = typeof users.$inferSelect;
type NewUser = typeof users.$inferInsert;

// Database client
const pool = new Pool({ connectionString: process.env.DATABASE_URL });
const db = drizzle(pool);

// Type-safe queries
async function getUserById(id: string): Promise<User | undefined> {
  const result = await db.select().from(users).where(eq(users.id, id)).limit(1);
  return result[0];
}

async function createUser(userData: NewUser): Promise<User> {
  const result = await db.insert(users).values(userData).returning();
  return result[0];
}

// Relations
async function getUserWithPosts(userId: string) {
  return db.query.users.findFirst({
    where: eq(users.id, userId),
    with: {
      posts: true
    }
  });
}
```

---

## 33.6 Environment Variables Type Safety

Environment variables are typically loaded as strings, but TypeScript can provide type safety through validation and declaration files.

### Validation with Zod

```typescript
// config/env.ts
import { z } from 'zod';
import dotenv from 'dotenv';

dotenv.config();

const envSchema = z.object({
  NODE_ENV: z.enum(['development', 'production', 'test']).default('development'),
  PORT: z.string().transform(Number).default('3000'),
  DATABASE_URL: z.string().url(),
  JWT_SECRET: z.string().min(32),
  JWT_EXPIRES_IN: z.string().default('24h'),
  REDIS_URL: z.string().url().optional(),
  API_KEY: z.string().optional(),
  LOG_LEVEL: z.enum(['error', 'warn', 'info', 'debug']).default('info')
});

const parsedEnv = envSchema.safeParse(process.env);

if (!parsedEnv.success) {
  console.error('Invalid environment variables:', parsedEnv.error.errors);
  process.exit(1);
}

export const env = parsedEnv.data;
export type Env = z.infer<typeof envSchema>;
```

### Declaration File Approach

```typescript
// types/env.d.ts
declare global {
  namespace NodeJS {
    interface ProcessEnv {
      NODE_ENV: 'development' | 'production' | 'test';
      PORT: string;
      DATABASE_URL: string;
      JWT_SECRET: string;
      JWT_EXPIRES_IN: string;
      REDIS_URL?: string;
      API_KEY?: string;
      LOG_LEVEL?: 'error' | 'warn' | 'info' | 'debug';
    }
  }
}

export {};
```

### Configuration Module

```typescript
// config/index.ts
import { env } from './env.js';

export const config = {
  env: env.NODE_ENV,
  port: env.PORT,
  database: {
    url: env.DATABASE_URL
  },
  jwt: {
    secret: env.JWT_SECRET,
    expiresIn: env.JWT_EXPIRES_IN
  },
  redis: env.REDIS_URL ? { url: env.REDIS_URL } : null,
  apiKey: env.API_KEY,
  logLevel: env.LOG_LEVEL
} as const;

export type Config = typeof config;
```

---

## 33.7 Chapter Summary and Exercises

### Chapter Summary

This chapter covered TypeScript in Node.js backend development:

**Key Concepts:**

1. **Setup**: Modern Node.js uses ES modules with TypeScript. Tools like `tsx` provide fast development experience, while `tsc` handles production builds with proper declaration files.

2. **Express**: Typed Request and Response generics enable compile-time checking of route handlers, middleware, and controllers. Declaration merging extends Express types for custom properties like authentication.

3. **Fastify**: TypeBox integration provides single-source-of-truth for both JSON Schema validation and TypeScript types, eliminating duplication and ensuring runtime validation matches compile-time types.

4. **NestJS**: Built for TypeScript, uses decorators for metadata, dependency injection for loose coupling, and class-validator for DTO validation. Guards and interceptors provide cross-cutting concerns with full type safety.

5. **Database**: ORMs like Prisma generate TypeScript types from database schemas. Query builders like Drizzle offer lightweight alternatives with inferred types. Repository patterns abstract database operations behind typed interfaces.

6. **Environment**: Schema validation (Zod) ensures required environment variables exist and have correct types before application startup, preventing runtime errors from missing configuration.

### Practical Exercises

**Exercise 1: Express REST API**

Build a complete REST API for a Task Manager:
- CRUD operations for Tasks (title, description, status, priority, dueDate)
- Validation middleware using Zod
- Error handling middleware with typed error responses
- Authentication middleware with JWT
- Type-safe request/response for all routes
- Pagination and filtering with typed query parameters

**Exercise 2: Fastify with TypeBox**

Create a Blog API using Fastify:
- Define schemas for Post, Comment, and User with TypeBox
- Implement CRUD with automatic validation
- Add authentication hook with typed request.user
- Create plugin for database connection with decorators
- Implement rate limiting with typed options
- Add response caching with proper cache headers

**Exercise 3: NestJS Microservice**

Build a User Service with NestJS:
- Modular architecture with UsersModule, AuthModule, and DatabaseModule
- Implement JWT strategy with Passport
- Create custom decorators (@CurrentUser, @Roles)
- Implement interceptors for logging and transformation
- Add exception filters for consistent error responses
- Write unit tests for services with Jest

**Exercise 4: Database Integration**

Set up Prisma with PostgreSQL:
- Define schema for E-commerce (Products, Orders, OrderItems, Users)
- Implement repository pattern abstraction
- Create typed service layer with transaction support
- Add soft delete and audit logging
- Implement complex queries with relations (user orders with products)
- Migration strategy for schema changes

**Exercise 5: Full-Stack Type Safety**

Create a monorepo with shared types:
- Set up pnpm workspace with frontend (React) and backend (Express)
- Define shared API types in common package
- Implement type-safe API client using fetch with shared types
- Create typed hooks for data fetching (useQuery, useMutation)
- Add WebSocket support with typed events
- Implement end-to-end type checking across frontend/backend boundary

**Exercise 6: Testing**

Write comprehensive tests:
- Unit tests for services with mocked dependencies
- Integration tests for API endpoints with test database
- E2E tests covering full request/response cycle
- Type testing with tsd to ensure type definitions are correct
- Load testing with typed metrics collection

### Additional Resources

- **Express TypeScript Documentation**: https://expressjs.com/en/starter/typescript.html
- **Fastify Documentation**: https://www.fastify.io/docs/latest/Reference/TypeScript/
- **NestJS Documentation**: https://docs.nestjs.com/
- **Prisma Documentation**: https://www.prisma.io/docs/concepts/components/prisma-client
- **Node.js Best Practices**: https://github.com/goldbergyoni/nodebestpractices

---

## Coming Up Next: Chapter 34 - TypeScript with Vue.js

In the next chapter, we will explore integrating TypeScript with Vue.js:

- Setting up Vue 3 projects with TypeScript (Vite vs Vue CLI)
- Composition API with TypeScript (ref, reactive, computed)
- Typing component props with defineProps
- Typing component emits with defineEmits
- Type-safe provide/inject
- Typing Vuex/Pinia stores
- Script setup syntax and type inference
- Generic components in Vue
- Type-safe router (Vue Router 4)

Understanding TypeScript with Vue.js enables you to build type-safe, maintainable frontend applications with excellent developer experience in the Vue ecosystem, leveraging the full power of Vue 3's Composition API with compile-time type checking.